# 检索分割好的文档，并进行回答

In [1]:
import shutup
shutup.please()

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model='qwen2:7b')

In [2]:
# 1. 加载并分割文档
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('rag_example/about.pdf')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20
)
documents = loader.load_and_split(text_splitter)
len(documents)

10

In [5]:
# 2. 召回相似片段
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

embedding = OpenAIEmbeddings(model='text-embedding-ada-002')

documents_db = FAISS.from_documents(documents, embedding)

question = '剧集主要角色包括?'
retrieval_ducuments = documents_db.similarity_search(question, k=3)
retrieval_ducuments

[Document(page_content='雄联盟宇宙中关于权力与自由的讨论。\n角色与制作\n剧集主要角色包括：\n蔚（Vi）： 执着且勇敢的地下拳手。\n金克丝（Jinx）： 具有不稳定精神状态的疯狂发明家。', metadata={'source': 'rag_example/about.pdf', 'file_path': 'rag_example/about.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': 'a', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/131.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m131', 'creationDate': "D:20241129070925+00'00'", 'modDate': "D:20241129070925+00'00'", 'trapped': ''}),
 Document(page_content='亚等。\n观众评价\n第二季播出后在全球范围内引发热烈讨论，其精美的动画制作、紧凑的剧情和对角色内心的刻画收获了\n观众的高度评价。不过，大结局也引发了一些争议，部分观众对剧情发展表达了不同意见。', metadata={'source': 'rag_example/about.pdf', 'file_path': 'rag_example/about.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': 'a', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/131.0.0.0 S

In [6]:
# 3. 基于召回片段问答
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm=llm, chain_type='stuff', verbose=True)
result = chain.run(input_documents=retrieval_ducuments, question=question)
result



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
雄联盟宇宙中关于权力与自由的讨论。
角色与制作
剧集主要角色包括：
蔚（Vi）： 执着且勇敢的地下拳手。
金克丝（Jinx）： 具有不稳定精神状态的疯狂发明家。

亚等。
观众评价
第二季播出后在全球范围内引发热烈讨论，其精美的动画制作、紧凑的剧情和对角色内心的刻画收获了
观众的高度评价。不过，大结局也引发了一些争议，部分观众对剧情发展表达了不同意见。

情感张力： 角色间的互动和冲突极具感染力，情节紧张且深刻。
伏笔与彩蛋： 每一集都埋藏了与英雄联盟世界观相关的细节，吸引了大批游戏粉丝。
未来计划
Human: 剧集主要角色包括?

> Finished chain.

> Finished chain.


'剧集的主要角色包括：\n\n1. **蔚（Vi）**：一个执着且勇敢的地下拳手。\n2. **金克丝（Jinx）**：一位具有不稳定精神状态的疯狂发明家。'